In [2]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [1]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00


In [8]:
from sklearn import datasets

In [13]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep= ";")

In [14]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [15]:
train,test= train_test_split(data, test_size= 0.25, random_state=42)


In [17]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8])

In [52]:
train_x= train.drop(["quality"], axis=1).values
train_y= train["quality"].values.ravel()
##test data
test_x= test.drop(["quality"], axis=1).values
test_y= test["quality"].values.ravel()
##split train data into train and validation data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.25, random_state=42)
signature = infer_signature(train_x, train_y)

In [82]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
  mean = np.mean(train_x, axis=0)
  var = np.var(train_x, axis = 0)

  model = keras.Sequential(
      [
          keras.Input([train_x.shape[1]]),
          keras.layers.Normalization(mean=mean, variance= var),
          keras.layers.Dense(64, activation="relu"),
          keras.layers.Dense(1)
      ]
  )

  model.compile(optimizer= keras.optimizers.SGD(
      learning_rate= params["lr"], momentum= params["momentum"]
  ), loss= "mean_squared_error",
                metrics= [keras.metrics.RootMeanSquaredError()])

  with mlflow.start_run(nested= True):
    model.fit(train_x, train_y, epochs= epochs, validation_data= (valid_x, valid_y), batch_size= 64)
    eval_result= model.evaluate(valid_x,valid_y, batch_size= 64)
    eval_rmse= eval_result[1]
    mlflow.log_params(params)
    mlflow.log_metric("eval_rmse", eval_rmse)

    mlflow.tensorflow.log_model(model, "model", signature= signature)
    return {"loss": eval_rmse, "status": STATUS_OK, "model": model}




In [83]:
def objective(params):
  result = train_model(
      params= params,
      epochs= 10,
      train_x= train_x,
      train_y= train_y,
      valid_x= valid_x,
      valid_y= valid_y,
      test_x= test_x,
      test_y= test_y
  )
  return result


In [84]:
space ={
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

In [85]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
  trials = Trials()
  best=fmin(
      fn= objective,
      space= space,
      algo= tpe.suggest,
      max_evals= 4,
      trials= trials
  )
  best_run = sorted(trials.results, key= lambda x: x["loss"])[0]
  mlflow.log_params(best)
  mlflow.log_metric("eval_rmse", best_run["loss"])
  mlflow.tensorflow.log_model(best_run['model'], "model", signature= signature)

  print(f"best parameters: {best}")
  print(f"best eval rmse: {best_run['loss']}")

Epoch 1/10

 1/44 ━━━━━━━━━━━━━━━━━━━━ 36s 838ms/step - loss: 40.4529 - root_mean_squared_error: 6.3603
25/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41.3998 - root_mean_squared_error: 6.4342   
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 41.2794 - root_mean_squared_error: 6.4249 - val_loss: 40.5289 - val_root_mean_squared_error: 6.3662

Epoch 2/10

 1/44 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 39.7496 - root_mean_squared_error: 6.3047
29/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40.5116 - root_mean_squared_error: 6.3648 
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.4394 - root_mean_squared_error: 6.3592 - val_loss: 39.8960 - val_root_mean_squared_error: 6.3163

Epoch 3/10

 1/44 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - loss: 39.9996 - root_mean_squared_error: 6.3245
25/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.6213 - root_mean_squared_error: 6.2945  
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39.6154 - root_mean_squared_error: 6.2941 - val_loss: 39.2694 - val_root_me

In [56]:
mlflow.set_tracking_uri("https://28cf-34-125-9-182.ngrok-free.app/")

In [41]:
!pip install mlflow --quiet
get_ipython().system_raw("mlflow ui --port 5000 &")

In [46]:
!pip install pyngrok --quiet

from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken: ··········
MLflow Tracking UI: https://28cf-34-125-9-182.ngrok-free.app
